In [1]:
# Import Essential Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style= "darkgrid", color_codes = True)
import warnings
warnings.filterwarnings('ignore')

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\KIIT\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py)

In [ ]:
# Import Dataset
loan_dataset= pd.read_csv("/kaggle/input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv")
loan_dataset.head(5)

In [ ]:
# More details about dataset
loan_dataset.info()

In [ ]:
loan_dataset.describe().T

In [ ]:
# Remove duplicates based on all columns
print(f'Shape of Loan Dataset before drop duplicated Row is: {loan_dataset.shape}')
loan_dataset = loan_dataset.drop_duplicates()
print(f'Shape of Loan Dataset After Drop Duplicated Row is: {loan_dataset.shape}')


In [ ]:
# Drop Unecessary Variables 
loan_dataset = loan_dataset.drop("Loan_ID", axis=1)

In [ ]:
# Categorical and Continuous Features

categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed','Property_Area','Credit_History']
continuous_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']

In [ ]:
# Splitting Data into Train, Validation, and Test Sets
X = loan_dataset.drop(columns='Loan_Status')
Y = loan_dataset['Loan_Status']

# Splitting into Train and Validation Sets
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2,stratify=Y, shuffle=True, random_state = 40)

# Splitting Validation Set into Validation and Test Sets
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, stratify=y_val, shuffle=True, random_state = 40)

In [ ]:

class Plotter:
    
    def __init__(self, X, y):
        self.x_train = X
        self.y_train = y
        self.fig, self.axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))  
    
    def Plot_Box(self, row, col, x):
        ax = self.axes[row, col]  
        sns.boxplot(data=self.x_train, x=x, y=self.y_train, ax=ax)
        ax.set_title(f"{x.capitalize()} vs. Loan_Status ")

    def Show_Plots(self):
        plt.tight_layout()
        plt.show()


In [ ]:
plot_continuous_features = Plotter(X_train, y_train)

# Plot the box plots
for i, col in enumerate(continuous_features):
    plot_continuous_features.Plot_Box(i // 2, i % 2, x=col)

plot_continuous_features.Show_Plots()

In [ ]:
# Create an DataFrame for Analysis Categorical Features 
result_df = pd.DataFrame(columns=['Features', 'Category', 'Count'])

for col in categorical_features:
    # Get the value counts for the column
    value_counts = loan_dataset[col].value_counts().reset_index()
    value_counts.columns = ['Category', 'Count']
    value_counts['Features'] = col
    result_df = pd.concat([result_df, value_counts], ignore_index=True)
    
# Display the result
grouped_df = result_df.groupby('Features').agg({'Category': list, 'Count': list})
grouped_df

In [ ]:
# Missing Value Analysis 
def calculate_missing_values(X_train, X_val, X_test):
    Miss_Train = X_train.isna().sum()
    Miss_Val = X_val.isna().sum()
    Miss_test = X_test.isna().sum()
    
    # Convert the series to dataframes
    output_train = pd.DataFrame(Miss_Train, columns=['Missing Values X_train'])
    output_val = pd.DataFrame(Miss_Val, columns=['Missing Values X_val'])
    output_test = pd.DataFrame(Miss_test, columns=['Missing Values X_test'])


    # Concatenate the dataframes output_train and output_val
    output = pd.concat([output_train, output_val,output_test], axis=1, join='inner')

    return output

output = calculate_missing_values(X_train, X_val, X_test)


# Define a function to apply the desired styling
def color_cell(value):
    if value >=20 :
        return 'background-color:#2e9ee8'
    elif value >=10 and value <20 :
        return 'background-color:#7ac1f0'
    elif value >=1 and value <10 :
        return 'background-color:#bdddf2'
    return ''

# Apply the styling to the DataFrame
styled_df = output.style.applymap(color_cell)

styled_df

In [ ]:
# Finding Rows with 50% or More Null Values in each sets

def find_rows_with_high_null_values(df):
    threshold = 0.5
    # Getting DataFrame Name
    df_name = [name for name in globals() if globals()[name] is df][0]
    null_threshold = int(threshold * len(df.columns))
    null_rows = df[df.apply(lambda x: x.isnull().sum(), axis=1) >= null_threshold]
    num_null_rows = len(null_rows)
    print(f"The number of rows consisting of more than 50% missing values in {df_name} is: {num_null_rows}")

# Example usage:
find_rows_with_high_null_values(X_train)
find_rows_with_high_null_values(X_val)
find_rows_with_high_null_values(X_test)

In [ ]:
# calculate mean columns

def calculate_mean(df, column):
    mean = df[column].mean().round()
    return mean

for col in ["LoanAmount","Loan_Amount_Term"]:
    print(f'Mean {col} in Trainset is: {calculate_mean(X_train, col)}')
    print(f'Mean {col} in Valset is: {calculate_mean(X_val, col)}')
    print(f'Mean {col} in Testset is: {calculate_mean(X_test, col)}')


In [ ]:
def fill_missing_values_by_mean(df, column):
    for col in cols:
        df[col].fillna(df[col].mean(), inplace=True)
    return df

cols = ["LoanAmount","Loan_Amount_Term"]
X_train = fill_missing_values_by_mean(X_train, cols)
X_val = fill_missing_values_by_mean(X_val, cols)
X_test = fill_missing_values_by_mean(X_test, cols)

In [ ]:
Isna_cate = ['Gender', 'Married','Education', 'Dependents', 'Self_Employed', 'Property_Area', 'Credit_History']

In [ ]:
# Most frequent value for each categorical feature
def Get_Mode(df, cols):
    list_of_most_frequent = {}
    for col in cols:
        f = df[col].mode().iloc[0]
        list_of_most_frequent[col] = f
    return pd.DataFrame(list_of_most_frequent, index=['Most Frequent'])


In [ ]:
# Most frequent value for each categorical feature in X_train
Get_Mode(X_train, Isna_cate)

In [ ]:
# Most frequent value for each categorical feature in X_Val
Get_Mode(X_val, Isna_cate)

In [ ]:
# Most frequent value for each categorical feature in X_test
Get_Mode(X_test, Isna_cate)

In [ ]:
#Handling Missing Values in Categorical Features

def Fill_Null(df,cols):
    
    for col in cols:
        df[col].fillna(df[col].mode()[0], inplace=True)

    return df

X_train = Fill_Null(X_train, Isna_cate)
X_val = Fill_Null(X_val, Isna_cate)
X_test = Fill_Null(X_test, Isna_cate)

In [ ]:
# Changing the data type of Credit_History to int using astype()
for df in [X_train, X_val, X_test]:
    df['Credit_History'] = df['Credit_History'].astype(int)

In [ ]:
X_train.isna().sum()

In [ ]:
X_val.isna().sum()

In [ ]:
X_test.isna().sum()

In [ ]:
Nominal_fetaures = ['Gender', 'Married','Education', 'Self_Employed', 'Property_Area']

In [ ]:
# Encoding Categorical Features
def encode_categorical_features(df, cols):
    
    for col in cols:
        dummies = pd.get_dummies(df[col], dtype=int, prefix=col)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(labels=col, axis=1)
    return df

# Encode the categorical features in the training and test sets
X_train = encode_categorical_features(X_train, Nominal_fetaures)
X_val = encode_categorical_features(X_val, Nominal_fetaures )
X_test = encode_categorical_features(X_test, Nominal_fetaures )

In [ ]:
# Encoding Categorical Features
encoder = {'0': 1/4, '1': 2/4, '2': 3/4, '3+': 4/4}
for df in [X_train, X_val, X_test]:
    df["Dependents"] = df['Dependents'].map(encoder)


In [ ]:
X_train.head(5)

In [ ]:
# Convet target column to int
mapping = {'Y': 1,'N': 0}
y_train = pd.Series(y_train).map(mapping)
y_val = pd.Series(y_val).map(mapping)
y_test = pd.Series(y_test).map(mapping)

In [ ]:
# Using SMOTE Technique  to Balancing TrainSet
# First figure
plt.subplot(1, 2, 1)
sns.countplot(x=y_train, data=X_train, palette='hls')
plt.ylabel('Loan Status')
plt.xlabel('Total')
plt.title('Original Data')

# Second figure
X_train, y_train = SMOTE().fit_resample(X_train, y_train)
plt.subplot(1, 2, 2)
sns.countplot(x=y_train, data=X_train, palette='hls')
plt.ylabel('Loan Status')
plt.xlabel('Total')
plt.title('SMOTE Resampled Data')

# Adjust the layout and display the figures
plt.tight_layout()
plt.show()

In [ ]:
# Correlation Analysis
train_copy = pd.concat([X_train, y_train], axis=1)
corr_matrix =train_copy.corr()
fig, ax = plt.subplots(figsize=(30, 20))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, ax=ax)
ax.set_title('Correlation Matrix')
plt.show()

In [ ]:
# Mutual Information
for X, y in zip([X_train, X_val, X_test], [y_train, y_val, y_test]):
    feature_names = X.columns
    scores = mutual_info_classif(X, y)

    # Create an array of indices for sorting the scores and feature names
    indices = np.argsort(scores)[::-1]

    # Sort the scores and feature names based on the indices
    sorted_scores = scores[indices]
    sorted_feature_names = [feature_names[i] for i in indices]

    # Plot the bar chart
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(sorted_scores)), sorted_scores)
    plt.xticks(range(len(sorted_scores)), sorted_feature_names, rotation='vertical')
    plt.title("Mutual Information Scores")
    plt.xlabel("Features")
    plt.ylabel("Score")
    plt.tight_layout()
    plt.show()

In [ ]:
# Dropping Features with 0.0 scores in MI

drop_fetaures = ['Dependents', 'CoapplicantIncome','Loan_Amount_Term', 'Gender_Female', 'Married_Yes',
                 'Education_Graduate', 'Self_Employed_No', 'Self_Employed_Yes','Property_Area_Urban']

def Dropping(df, cols):
    
    for col in cols:
        df = df.drop(labels=col, axis=1)
    return df

X_train = Dropping(X_train, drop_fetaures)
X_val = Dropping(X_val, drop_fetaures )
X_test = Dropping(X_test, drop_fetaures )

In [ ]:
# Scalling the dataset using StandardScaler
scaler = StandardScaler()

X_V = X_val.values
X_T = X_test.values
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_val = scaler.transform(X_V)
scaled_x_test = scaler.transform(X_T)

In [ ]:
# Dimensionality Reduction with PCA
pca = PCA(n_components = 7)
train_reduced = pca.fit_transform(scaled_x_train)
val_reduced = pca.transform(scaled_x_val)
test_reduced = pca.transform(scaled_x_test)


In [ ]:
# Import Dataset
loan_dataset= pd.read_csv("/kaggle/input/loan-predication/train_u6lujuX_CVtuZ9i (1).csv")

In [ ]:
# Drop Unecessary Variables 
loan_dataset = loan_dataset.drop("Loan_ID", axis=1)

In [ ]:
# Splitting Data into Train, Validation, and Test Sets
X = loan_dataset.drop(columns='Loan_Status')
Y = loan_dataset['Loan_Status']

# Splitting into Train and Validation Sets
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.2,stratify=Y, shuffle=True, random_state = 40)

# Splitting Validation Set into Validation and Test Sets
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, stratify=y_val, shuffle=True, random_state = 40)

In [ ]:
class Preprocessor:
    def __init__(self, X_train, X_val, X_test, categorical_features, continuous_features):
        self.X_train = X_train
        self.X_val = X_val
        self.X_test = X_test
        self.categorical_features = categorical_features
        self.continuous_features = continuous_features
        self.scaler = StandardScaler()
        self.pca = PCA(n_components=7)

    def handle_missing_values_cont(self):
        for col in self.continuous_features:
            self.X_train[col].fillna(self.X_train[col].mean(), inplace=True)
            self.X_val[col].fillna(self.X_val[col].mean(), inplace=True)
            self.X_test[col].fillna(self.X_test[col].mean(), inplace=True)
        print("Missing values in continuous columns handled.")
        return self.X_train, self.X_val ,self.X_test



    def handle_missing_values_cate(self):
        for col in self.categorical_features:
            self.X_train[col].fillna(self.X_train[col].mode()[0], inplace=True)
            self.X_val[col].fillna(self.X_val[col].mode()[0], inplace=True)
            self.X_test[col].fillna(self.X_test[col].mode()[0], inplace=True)
        print("Missing values in categorical columns handled.")
        return self.X_train, self.X_val ,self.X_test

    def encode_categorical_features(self):
        for col in self.categorical_features:
            if col == 'Dependents':
                encoder = {'0': 1/4, '1': 2/4, '2': 3/4, '3+': 4/4}
                self.X_train[col] = self.X_train[col].map(encoder)
                self.X_val[col] = self.X_val[col].map(encoder)
                self.X_test[col] = self.X_test[col].map(encoder)
            else:
                train_dummies = pd.get_dummies(self.X_train[col], dtype=int, prefix=col)
                val_dummies = pd.get_dummies(self.X_val[col], dtype=int, prefix=col)
                test_dummies = pd.get_dummies(self.X_test[col], dtype=int, prefix=col)
                self.X_train = pd.concat([self.X_train, train_dummies], axis=1).drop(labels=col, axis=1)
                self.X_val = pd.concat([self.X_val, val_dummies], axis=1).drop(labels=col, axis=1)
                self.X_test = pd.concat([self.X_test, test_dummies], axis=1).drop(labels=col, axis=1)
        print("Categorical features encoded.")
        
        return self.X_train, self.X_val ,self.X_test

    def Scaling(self):
        self.V = self.X_val.values
        self.T = self.X_test.values
        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_val = self.scaler.transform(self.V)
        self.X_test = self.scaler.transform(self.T)
        print("All DataSets Scaled!")
        return self.X_train, self.X_val ,self.X_test

    def DR(self):
        self.X_train = self.pca.fit_transform(self.X_train)
        self.X_val = self.pca.transform(self.X_val)
        self.X_test = self.pca.transform(self.X_test)
        print("Dimensionality reduction (PCA) applied on All DataSet")
        return self.X_train, self.X_val ,self.X_test


In [ ]:
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']
continuous_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']

preprocessor = Preprocessor(X_train,X_val,X_test ,categorical_features, continuous_features)


no_miss_X_train, no_miss_X_val ,no_miss_X_test = preprocessor.handle_missing_values_cont()
no_miss_X_train, no_miss_X_val ,no_miss_X_test = preprocessor.handle_missing_values_cate()
preprocessed_X_train, preprocessed_X_val ,preprocessed_X_test = preprocessor.encode_categorical_features()
scaled_X_train, scaled_X_val ,scaled_X_test = preprocessor.Scaling()
reduced_X_train, reduced_X_val ,reduced_X_test = preprocessor.DR()



In [ ]:
preprocessed_X_train.to_csv('TrainSet.csv', index=False)
preprocessed_X_val.to_csv('ValidationSet.csv', index=False)
preprocessed_X_test.to_csv('TestSet.csv', index=False)